> CNN512 With Dropout Luminosity

In [30]:
import pandas as pd
import numpy as np
import cv2
from skimage import exposure
from scipy.ndimage import gaussian_filter
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import models, layers
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.callbacks import LearningRateScheduler
import math
from sklearn.utils.class_weight import compute_class_weight
import numpy as np
import pandas as pd
import cv2
from skimage import exposure
from scipy.ndimage import gaussian_filter
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

In [31]:
# Load the dataset
data = pd.read_csv("test_split_image_key.csv")

# Split the dataset into training and testing sets
train_data = data[data["split_type"] == "training"]
test_data = data[data["split_type"] == "testing"]

# Define output directories for preprocessed images
output_dir_train = "preprocessed_images_luminosity/train/"
output_dir_test = "preprocessed_images_luminosity/test/"

# Create directories if they don't exist
os.makedirs(output_dir_train, exist_ok=True)
os.makedirs(output_dir_test, exist_ok=True)

In [32]:
# Preprocessing function
def preprocess_image(image_path, output_dir):
    # Load and resize the image to 512x512
    image = cv2.imread(image_path)
    resized_image = cv2.resize(image, (512, 512))

    # Enhance luminosity and apply Contrast Limited Adaptive Histogram Equalization (CLAHE)
    lab = cv2.cvtColor(resized_image, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab)
    clahe = cv2.createCLAHE(clipLimit=5.0, tileGridSize=(8, 8))
    l_eq = clahe.apply(l)
    lab_eq = cv2.merge((l_eq, a, b))
    enhanced_image = cv2.cvtColor(lab_eq, cv2.COLOR_LAB2BGR)

    # Image noise removal using Gaussian filter
    filtered_image = gaussian_filter(enhanced_image, sigma=1)

    # Save the preprocessed image
    image_name = os.path.basename(image_path)
    output_path = os.path.join(output_dir, image_name)
    cv2.imwrite(output_path, filtered_image)


# Apply preprocessing to training and testing images
for img_name in train_data["Image name"]:
    image_path = "a_training/" + img_name + ".jpg"
    preprocess_image(image_path, output_dir_train)

for img_name in test_data["Image name"]:
    image_path = "b_testing/" + img_name + ".jpg"
    preprocess_image(image_path, output_dir_test)

In [33]:
# Convert the images to numpy arrays
X_train = []
X_test = []

for img_name in train_data["Image name"]:
    image_path = os.path.join(output_dir_train, img_name + ".jpg")
    image = cv2.imread(image_path)
    X_train.append(image)

for img_name in test_data["Image name"]:
    image_path = os.path.join(output_dir_test, img_name + ".jpg")
    image = cv2.imread(image_path)
    X_test.append(image)

X_train = np.array(X_train)
X_test = np.array(X_test)

# Extract labels directly without one-hot encoding
y_train = train_data["Retinopathy grade"].values
y_test = test_data["Retinopathy grade"].values

In [34]:
input_shape = (512, 512, 3)
num_classes = 5
epochs = 50  # placeholder

In [35]:
cnn512_dropout = models.Sequential(
    [
        keras.Input(shape=input_shape),
        # Input Layer	Zero Padding layer	Padding (2,2)
        layers.ZeroPadding2D(padding=(2, 2)),
        # Layer 1	2D CONV layer	Kernel number = 32, kernel size = 3
        layers.Conv2D(32, kernel_size=(3, 3)),
        # Layer 2	Batch Normalization layer	-
        layers.BatchNormalization(),
        # Layer 3	Relu layer	-
        layers.Activation("relu"),
        # Layer 4	Max Pooling layer	Pooling size (2,2)
        layers.MaxPooling2D(pool_size=(2, 2)),
        # Layer 5	2D CONV layer	Kernel number = 64, kernel size = 3
        layers.Conv2D(64, kernel_size=(3, 3)),
        # Layer 6	Batch Normalization layer	-
        layers.BatchNormalization(),
        # Layer 7	Relu layer	-
        layers.Activation("relu"),
        # Layer 8	Max Pooling layer	Pooling size (2, 2)
        layers.MaxPooling2D(pool_size=(2, 2)),
        # Layer 9	2D CONV layer	Kernel number = 96, kernel size = 3
        layers.Conv2D(96, kernel_size=(3, 3)),
        # Layer 10	Batch Normalization layer	-
        layers.BatchNormalization(),
        # Layer 11	Relu layer	-
        layers.Activation("relu"),
        # Layer 12	Max Pooling layer	Pooling size (2, 2)
        layers.MaxPooling2D(pool_size=(2, 2)),
        # Layer 13	2D CONV layer	Kernel number = 96, kernel size = 3
        layers.Conv2D(96, kernel_size=(3, 3)),
        # Layer 14	Batch Normalization layer	-
        layers.BatchNormalization(),
        # Layer 15	Relu layer	-
        layers.Activation("relu"),
        # Layer 16	Max Pooling layer	Pooling size (2, 2)
        layers.MaxPooling2D(pool_size=(2, 2)),
        # Layer 17	2D CONV layer	Kernel number = 128, kernel size = 3
        layers.Conv2D(128, kernel_size=(3, 3)),
        # Layer 18	Batch Normalization layer	-
        layers.BatchNormalization(),
        # Layer 19	Relu layer	-
        layers.Activation("relu"),
        # Layer 20	Max Pooling layer	Pooling size (2, 2)
        layers.MaxPooling2D(pool_size=(2, 2)),
        # Layer 21	2D CONV layer	Kernel number = 200, kernel size = 3
        layers.Conv2D(200, kernel_size=(3, 3)),
        # Layer 22	Batch Normalization layer	-
        layers.BatchNormalization(),
        # Layer 23	Relu layer	-
        layers.Activation("relu"),
        # Layer 24	Max Pooling layer	Pooling size (2, 2)
        layers.MaxPooling2D(pool_size=(2, 2)),
        # Layer 25	Flatten layer	-
        layers.Flatten(),
        # Layer 26	FC layer	Neurons number = 1000
        layers.Dense(1000),
        # Layer 27	Batch Normalization layer	-
        layers.BatchNormalization(),
        # Layer 28	Relu layer	-
        layers.Activation("relu"),
        layers.Dropout(0.5),
        # Layer 29	FC layer	Neurons number = 500
        layers.Dense(500),
        # Layer 30	Batch Normalization layer	-
        layers.BatchNormalization(),
        # Layer 31	Relu layer	-
        layers.Activation("relu"),
        layers.Dropout(0.5),
        # Layer 32	FC layer with Softmax activatio
        layers.Dense(num_classes, activation="softmax"),
    ]
)

In [36]:
from tensorflow.keras.optimizers import SGD

# base learning rate for custom CNNs
base_learning_rate = 1e-4
# maximum learning rate for custom CNNs
max_learning_rate = 1e-1

# Create an instance of SGD optimizer with initial learning rate
optimizer = SGD(learning_rate=base_learning_rate, momentum=0.9)

cnn512_dropout.compile(
    optimizer=optimizer, loss="sparse_categorical_crossentropy", metrics=["accuracy"]
)

# create class weight
classes = np.unique(y_train)
class_weights = compute_class_weight("balanced", classes=classes, y=y_train)
class_weight_dict = dict(zip(classes, class_weights))


# create triangular schedule
def triangular_schedule(epoch):
    """Triangular learning rate scheduler."""
    cycle_length = 10  # Define the length of a cycle
    cycle = math.floor(1 + epoch / (2 * cycle_length))
    x = abs(epoch / cycle_length - 2 * cycle + 1)
    lr = base_learning_rate + (max_learning_rate - base_learning_rate) * max(0, (1 - x))
    return lr


# When fitting the model, include the learning rate scheduler callback
lr_scheduler = LearningRateScheduler(triangular_schedule)

In [37]:
# fitting the model
history_cnn512_dropout = cnn512_dropout.fit(
    X_train,
    y_train,
    epochs=epochs,
    batch_size=32,
    class_weight=class_weight_dict,
    callbacks=[lr_scheduler],
)

Epoch 1/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 161s 11s/step - accuracy: 0.1982 - loss: 2.2695 - learning_rate: 1.0000e-04
Epoch 2/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 185s 14s/step - accuracy: 0.1919 - loss: 2.1330 - learning_rate: 0.0101
Epoch 3/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 169s 13s/step - accuracy: 0.3296 - loss: 1.9230 - learning_rate: 0.0201
Epoch 4/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 180s 13s/step - accuracy: 0.3588 - loss: 2.0680 - learning_rate: 0.0301
Epoch 5/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 191s 15s/step - accuracy: 0.3335 - loss: 2.1243 - learning_rate: 0.0401
Epoch 6/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 185s 14s/step - accuracy: 0.3214 - loss: 2.6462 - learning_rate: 0.0501
Epoch 7/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 205s 16s/step - accuracy: 0.2674 - loss: 3.6063 - learning_rate: 0.0600
Epoch 8/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 160s 12s/step - accuracy: 0.2861 - loss: 3.6982 - learning_rate: 0.0700
Epoch 9/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 193s 15s/step - accuracy: 0.2331 - loss: 4.7541 - learning_rate: 0.0800
Epoch 

In [38]:
# Evaluate the model
test_loss, test_accuracy = cnn512_dropout.evaluate(X_test, y_test)
print("Test Accuracy:", test_accuracy)

4/4 ━━━━━━━━━━━━━━━━━━━━ 6s 1s/step - accuracy: 0.2960 - loss: 2.7192
Test Accuracy: 0.24271844327449799


> CNN512 With Dropout Luminosity + normalize by 225 + augmentaion

In [39]:
X_train = X_train.astype("float32") / 225  # normalize
X_test = X_test.astype("float32") / 225  # normalize

In [40]:
cnn512_dropout = models.Sequential(
    [
        keras.Input(shape=input_shape),
        # Input Layer	Zero Padding layer	Padding (2,2)
        layers.ZeroPadding2D(padding=(2, 2)),
        # Layer 1	2D CONV layer	Kernel number = 32, kernel size = 3
        layers.Conv2D(32, kernel_size=(3, 3)),
        # Layer 2	Batch Normalization layer	-
        layers.BatchNormalization(),
        # Layer 3	Relu layer	-
        layers.Activation("relu"),
        # Layer 4	Max Pooling layer	Pooling size (2,2)
        layers.MaxPooling2D(pool_size=(2, 2)),
        # Layer 5	2D CONV layer	Kernel number = 64, kernel size = 3
        layers.Conv2D(64, kernel_size=(3, 3)),
        # Layer 6	Batch Normalization layer	-
        layers.BatchNormalization(),
        # Layer 7	Relu layer	-
        layers.Activation("relu"),
        # Layer 8	Max Pooling layer	Pooling size (2, 2)
        layers.MaxPooling2D(pool_size=(2, 2)),
        # Layer 9	2D CONV layer	Kernel number = 96, kernel size = 3
        layers.Conv2D(96, kernel_size=(3, 3)),
        # Layer 10	Batch Normalization layer	-
        layers.BatchNormalization(),
        # Layer 11	Relu layer	-
        layers.Activation("relu"),
        # Layer 12	Max Pooling layer	Pooling size (2, 2)
        layers.MaxPooling2D(pool_size=(2, 2)),
        # Layer 13	2D CONV layer	Kernel number = 96, kernel size = 3
        layers.Conv2D(96, kernel_size=(3, 3)),
        # Layer 14	Batch Normalization layer	-
        layers.BatchNormalization(),
        # Layer 15	Relu layer	-
        layers.Activation("relu"),
        # Layer 16	Max Pooling layer	Pooling size (2, 2)
        layers.MaxPooling2D(pool_size=(2, 2)),
        # Layer 17	2D CONV layer	Kernel number = 128, kernel size = 3
        layers.Conv2D(128, kernel_size=(3, 3)),
        # Layer 18	Batch Normalization layer	-
        layers.BatchNormalization(),
        # Layer 19	Relu layer	-
        layers.Activation("relu"),
        # Layer 20	Max Pooling layer	Pooling size (2, 2)
        layers.MaxPooling2D(pool_size=(2, 2)),
        # Layer 21	2D CONV layer	Kernel number = 200, kernel size = 3
        layers.Conv2D(200, kernel_size=(3, 3)),
        # Layer 22	Batch Normalization layer	-
        layers.BatchNormalization(),
        # Layer 23	Relu layer	-
        layers.Activation("relu"),
        # Layer 24	Max Pooling layer	Pooling size (2, 2)
        layers.MaxPooling2D(pool_size=(2, 2)),
        # Layer 25	Flatten layer	-
        layers.Flatten(),
        # Layer 26	FC layer	Neurons number = 1000
        layers.Dense(1000),
        # Layer 27	Batch Normalization layer	-
        layers.BatchNormalization(),
        # Layer 28	Relu layer	-
        layers.Activation("relu"),
        layers.Dropout(0.5),
        # Layer 29	FC layer	Neurons number = 500
        layers.Dense(500),
        # Layer 30	Batch Normalization layer	-
        layers.BatchNormalization(),
        # Layer 31	Relu layer	-
        layers.Activation("relu"),
        layers.Dropout(0.5),
        # Layer 32	FC layer with Softmax activatio
        layers.Dense(num_classes, activation="softmax"),
    ]
)

In [41]:
from tensorflow.keras.optimizers import SGD

# base learning rate for custom CNNs
base_learning_rate = 1e-4
# maximum learning rate for custom CNNs
max_learning_rate = 1e-1

# Create an instance of SGD optimizer with initial learning rate
optimizer = SGD(learning_rate=base_learning_rate, momentum=0.9)

cnn512_dropout.compile(
    optimizer=optimizer, loss="sparse_categorical_crossentropy", metrics=["accuracy"]
)

# create class weight
classes = np.unique(y_train)
class_weights = compute_class_weight("balanced", classes=classes, y=y_train)
class_weight_dict = dict(zip(classes, class_weights))


# create triangular schedule
def triangular_schedule(epoch):
    """Triangular learning rate scheduler."""
    cycle_length = 10  # Define the length of a cycle
    cycle = math.floor(1 + epoch / (2 * cycle_length))
    x = abs(epoch / cycle_length - 2 * cycle + 1)
    lr = base_learning_rate + (max_learning_rate - base_learning_rate) * max(0, (1 - x))
    return lr


# When fitting the model, include the learning rate scheduler callback
lr_scheduler = LearningRateScheduler(triangular_schedule)

# Define the data augmentation generator
data_augmentation = ImageDataGenerator(
    rotation_range=20,  # Random rotation between 0 and 20 degrees
    width_shift_range=0.2,  # Random horizontal shift
    height_shift_range=0.2,  # Random vertical shift
    shear_range=0.2,  # Shear transformation
    zoom_range=0.2,  # Random zoom
    horizontal_flip=True,  # Random horizontal flip
    fill_mode="nearest",  # Strategy for filling in newly created pixels
)

# Apply the data augmentation using .flow() and then fit the model
augmented_data = data_augmentation.flow(X_train, y_train, batch_size=32)

In [42]:
import math

# Fit the model with the augmented data generator
history_cnn512_dropout = cnn512_dropout.fit(
    augmented_data,
    steps_per_epoch=math.ceil(len(X_train) / 32),  # Round up to ensure it's an integer
    epochs=epochs,
    class_weight=class_weight_dict,
    callbacks=[lr_scheduler],
)

Epoch 1/50


/Users/tinayiluo/miniconda3/lib/python3.11/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


13/13 ━━━━━━━━━━━━━━━━━━━━ 203s 14s/step - accuracy: 0.2462 - loss: 2.3824 - learning_rate: 1.0000e-04
Epoch 2/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - learning_rate: 0.0101
Epoch 3/50


2024-04-02 23:45:47.797551: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
/Users/tinayiluo/miniconda3/lib/python3.11/contextlib.py:155: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


13/13 ━━━━━━━━━━━━━━━━━━━━ 228s 17s/step - accuracy: 0.2367 - loss: 2.3679 - learning_rate: 0.0201
Epoch 4/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - learning_rate: 0.0301
Epoch 5/50


...

13/13 ━━━━━━━━━━━━━━━━━━━━ 193s 14s/step - accuracy: 0.3721 - loss: 1.7789 - learning_rate: 0.0600
Epoch 48/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - learning_rate: 0.0700
Epoch 49/50


2024-04-03 01:07:08.551423: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


13/13 ━━━━━━━━━━━━━━━━━━━━ 203s 15s/step - accuracy: 0.2157 - loss: 1.8075 - learning_rate: 0.0800
Epoch 50/50


2024-04-03 01:10:31.864852: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 73ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - learning_rate: 0.0900


In [43]:
# Evaluate the model
test_loss, test_accuracy = cnn512_dropout.evaluate(X_test, y_test)
print("Test Accuracy:", test_accuracy)

4/4 ━━━━━━━━━━━━━━━━━━━━ 7s 1s/step - accuracy: 0.2938 - loss: 1.7852
Test Accuracy: 0.21359223127365112


> CNN512 Without Dropout Luminosity + normalization by 225

In [44]:
cnn512_nodropout = models.Sequential(
    [
        keras.Input(shape=input_shape),
        # Input Layer	Zero Padding layer	Padding (2,2)
        layers.ZeroPadding2D(padding=(2, 2)),
        # Layer 1	2D CONV layer	Kernel number = 32, kernel size = 3
        layers.Conv2D(32, kernel_size=(3, 3)),
        # Layer 2	Batch Normalization layer	-
        layers.BatchNormalization(),
        # Layer 3	Relu layer	-
        layers.Activation("relu"),
        # Layer 4	Max Pooling layer	Pooling size (2,2)
        layers.MaxPooling2D(pool_size=(2, 2)),
        # Layer 5	2D CONV layer	Kernel number = 64, kernel size = 3
        layers.Conv2D(64, kernel_size=(3, 3)),
        # Layer 6	Batch Normalization layer	-
        layers.BatchNormalization(),
        # Layer 7	Relu layer	-
        layers.Activation("relu"),
        # Layer 8	Max Pooling layer	Pooling size (2, 2)
        layers.MaxPooling2D(pool_size=(2, 2)),
        # Layer 9	2D CONV layer	Kernel number = 96, kernel size = 3
        layers.Conv2D(96, kernel_size=(3, 3)),
        # Layer 10	Batch Normalization layer	-
        layers.BatchNormalization(),
        # Layer 11	Relu layer	-
        layers.Activation("relu"),
        # Layer 12	Max Pooling layer	Pooling size (2, 2)
        layers.MaxPooling2D(pool_size=(2, 2)),
        # Layer 13	2D CONV layer	Kernel number = 96, kernel size = 3
        layers.Conv2D(96, kernel_size=(3, 3)),
        # Layer 14	Batch Normalization layer	-
        layers.BatchNormalization(),
        # Layer 15	Relu layer	-
        layers.Activation("relu"),
        # Layer 16	Max Pooling layer	Pooling size (2, 2)
        layers.MaxPooling2D(pool_size=(2, 2)),
        # Layer 17	2D CONV layer	Kernel number = 128, kernel size = 3
        layers.Conv2D(128, kernel_size=(3, 3)),
        # Layer 18	Batch Normalization layer	-
        layers.BatchNormalization(),
        # Layer 19	Relu layer	-
        layers.Activation("relu"),
        # Layer 20	Max Pooling layer	Pooling size (2, 2)
        layers.MaxPooling2D(pool_size=(2, 2)),
        # Layer 21	2D CONV layer	Kernel number = 200, kernel size = 3
        layers.Conv2D(200, kernel_size=(3, 3)),
        # Layer 22	Batch Normalization layer	-
        layers.BatchNormalization(),
        # Layer 23	Relu layer	-
        layers.Activation("relu"),
        # Layer 24	Max Pooling layer	Pooling size (2, 2)
        layers.MaxPooling2D(pool_size=(2, 2)),
        # Layer 25	Flatten layer	-
        layers.Flatten(),
        # Layer 26	FC layer	Neurons number = 1000
        layers.Dense(1000),
        # Layer 27	Batch Normalization layer	-
        layers.BatchNormalization(),
        # Layer 28	Relu layer	-
        layers.Activation("relu"),
        # layers.Dropout(0.5),
        # Layer 29	FC layer	Neurons number = 500
        layers.Dense(500),
        # Layer 30	Batch Normalization layer	-
        layers.BatchNormalization(),
        # Layer 31	Relu layer	-
        layers.Activation("relu"),
        # layers.Dropout(0.5),
        # Layer 32	FC layer with Softmax activatio
        layers.Dense(num_classes, activation="softmax"),
    ]
)

In [45]:
from tensorflow.keras.optimizers import SGD

# base learning rate for custom CNNs
base_learning_rate = 1e-4
# maximum learning rate for custom CNNs
max_learning_rate = 1e-1

# Create an instance of SGD optimizer with initial learning rate
optimizer = SGD(learning_rate=base_learning_rate, momentum=0.9)

cnn512_nodropout.compile(
    optimizer=optimizer, loss="sparse_categorical_crossentropy", metrics=["accuracy"]
)

# create class weight
classes = np.unique(y_train)
class_weights = compute_class_weight("balanced", classes=classes, y=y_train)
class_weight_dict = dict(zip(classes, class_weights))


# create triangular schedule
def triangular_schedule(epoch):
    """Triangular learning rate scheduler."""
    cycle_length = 10  # Define the length of a cycle
    cycle = math.floor(1 + epoch / (2 * cycle_length))
    x = abs(epoch / cycle_length - 2 * cycle + 1)
    lr = base_learning_rate + (max_learning_rate - base_learning_rate) * max(0, (1 - x))
    return lr


# When fitting the model, include the learning rate scheduler callback
lr_scheduler = LearningRateScheduler(triangular_schedule)

In [46]:
# fitting the model
history_cnn512_nodropout = cnn512_nodropout.fit(
    X_train,
    y_train,
    epochs=epochs,
    batch_size=32,
    class_weight=class_weight_dict,
    callbacks=[lr_scheduler],
)

Epoch 1/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 187s 14s/step - accuracy: 0.1585 - loss: 2.0560 - learning_rate: 1.0000e-04
Epoch 2/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 189s 14s/step - accuracy: 0.2368 - loss: 1.8535 - learning_rate: 0.0101
Epoch 3/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 186s 14s/step - accuracy: 0.4484 - loss: 1.4537 - learning_rate: 0.0201
Epoch 4/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 179s 14s/step - accuracy: 0.4428 - loss: 1.4507 - learning_rate: 0.0301
Epoch 5/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 198s 15s/step - accuracy: 0.4549 - loss: 1.6488 - learning_rate: 0.0401
Epoch 6/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 178s 13s/step - accuracy: 0.4944 - loss: 1.4461 - learning_rate: 0.0501
Epoch 7/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 175s 13s/step - accuracy: 0.5267 - loss: 1.7068 - learning_rate: 0.0600
Epoch 8/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 178s 13s/step - accuracy: 0.5272 - loss: 1.5202 - learning_rate: 0.0700
Epoch 9/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 183s 14s/step - accuracy: 0.6033 - loss: 1.4063 - learning_rate: 0.0800
Epoch 

In [47]:
# Evaluate the model
test_loss, test_accuracy = cnn512_nodropout.evaluate(X_test, y_test)
print("Test Accuracy:", test_accuracy)

4/4 ━━━━━━━━━━━━━━━━━━━━ 7s 1s/step - accuracy: 0.3969 - loss: 4.3096
Test Accuracy: 0.42718446254730225
